In [1]:
import pandas as pd

In [2]:
import numpy as np


    header = None — загрузить без строки с заголовком;
    skiprows=n — пропустить n строк (часто у документов бывает техническая «шапка»);
    encoding — загрузить в конкретной кодировке;
    na_values — список значений, который нужно заменить на NaN (специальный объект, обозначающий пропущенное значение).


In [35]:
log = pd.read_csv("data/log.csv", header = None)

In [36]:
users = pd.read_csv("data/users.csv", encoding="KOI8-R", sep="	")

In [37]:
log.columns = ['user_id','time','bet','win']

In [38]:
users.columns = ['user_id','email','geo']

In [39]:
# Приведём признак user_id к одному формату в обоих датасетах
users.user_id = users.user_id.apply(lambda x: x.lower())
# Избавимся от ошибок в user_id
log = log[log.user_id != "#error"]
log.user_id = log.user_id.str.split(" - ").apply(lambda x: x[1])

In [40]:
users_log = pd.merge(log, users, on='user_id')

В данном случае мы группируем данные по признаку user_id.

После этого в каждой группе выбираем признак win.

Затем берём медиану по каждой группе по признаку win и на выходе получаем таблицу, где индексом является признак user_id. В этой таблице единственный столбец — медиана по каждой группе (то есть по каждому пользователю).

Наконец, последний вызов median() даёт нам медиану по предыдущему столбцу, то есть возвращает одно число.

In [46]:
users_log.groupby('user_id').win.median().median() 

0.0

In [45]:
users_log['bet'] = users_log["bet"].fillna(0)
users_log['win'] = users_log["win"].fillna(0)

In [48]:
users_log['net'] = users_log.win - users_log.bet

In [22]:
percent_bet = users_log[users_log.bet > 0].bet.shape[0] / users_log.shape[0]
print('Процент ставок: {:.2%}'.format(percent_bet))

Процент ставок: 48.21%


In [49]:
users_log.groupby('user_id').net.sum().median()

1986.0

In [18]:
users_log.head()

,user_id,time,bet,win,email,geo,net
0,user_919,[2019-01-01 14:06:51,0.0,0.0,Chikkaverle@icloud.com,Хабаровск,0.0
1,user_919,[2019-01-30 10:06:00,0.0,0.0,Chikkaverle@icloud.com,Хабаровск,0.0
2,user_919,[2019-02-05 14:33:44,0.0,0.0,Chikkaverle@icloud.com,Хабаровск,0.0
3,user_919,[2019-02-14 11:38:05,0.0,0.0,Chikkaverle@icloud.com,Хабаровск,0.0
4,user_919,[2019-03-02 4:23:36,300.0,0.0,Chikkaverle@icloud.com,Хабаровск,-300.0


In [ ]:
log_win = log[log['win'] > 0]

In [ ]:
win_count = log_win.shape[0]
print(win_count)

In [ ]:
log2 = log.query("bet<2000 & win>0")

In [ ]:
log2

In [ ]:
log.user_id.unique()

In [171]:
log = log[~log.user_id.str.contains("error", na=False)]

In [172]:
def get_user_id(s):
    pos = s.find("user")
    if pos > -1:
        return s[pos:]
    else:
        return ""

In [173]:
log['user_id'] = log.user_id.apply(get_user_id)


In [174]:
#t = log.time[0]
log['time'] = log[log.time.str.match("\[", na=False)]['time'].apply(lambda t: str(t).replace('[', ''))
#t = t[1:]

In [175]:
log['time'] = pd.to_datetime(log['time'])

In [ ]:
np.isnan(log.time.values).sum()

In [176]:
log_backup = log.copy()

In [ ]:
log = log_backup.copy()

In [ ]:
log.isnull()

In [190]:
log.dropna(axis=0, subset=['user_id', 'time'])

,user_id,time,bet,win,daytime
0,user_919,2019-01-01 14:06:51,NaN,NaN,day
1,user_973,2019-01-01 14:51:16,NaN,NaN,day
2,user_903,2019-01-01 16:31:16,NaN,NaN,day
3,user_954,2019-01-01 17:17:51,NaN,NaN,day
4,user_954,2019-01-01 21:31:18,NaN,NaN,elvin
...,...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0,day
992,user_967,2019-04-20 14:59:36,10154.0,NaN,day
993,user_973,2019-04-20 17:09:56,10254.0,NaN,day
994,user_977,2019-04-20 18:10:07,10354.0,NaN,elvin


In [ ]:
log = log.drop_duplicates(subset=['user_id', 'time'])

In [ ]:
log.info()

In [ ]:
log.time.max()

In [193]:
log

,user_id,time,bet,win,daytime,hour
0,user_919,2019-01-01 14:06:51,NaN,NaN,day,14
1,user_973,2019-01-01 14:51:16,NaN,NaN,day,14
2,user_903,2019-01-01 16:31:16,NaN,NaN,day,16
3,user_954,2019-01-01 17:17:51,NaN,NaN,day,17
4,user_954,2019-01-01 21:31:18,NaN,NaN,elvin,21
...,...,...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0,day,12
992,user_967,2019-04-20 14:59:36,10154.0,NaN,day,14
993,user_973,2019-04-20 17:09:56,10254.0,NaN,day,17
994,user_977,2019-04-20 18:10:07,10354.0,NaN,elvin,18


In [187]:
def hour_to_daytime(hours):
    if hours >=0 and hours < 6:
        return 'night'
    elif hours >= 6 and hours < 12:
        return 'morning'
    elif hours >= 12 and hours < 18:
        return 'day'
    else:
        return 'elvin'

In [188]:
log['daytime'] = log.time.dt.hour.apply(hour_to_daytime)

In [189]:
log.daytime.value_counts(ascending=True)

elvin      227
day        240
morning    253
night      265
Name: daytime, dtype: int64

In [192]:
log['hour'] = log.time.dt.hour